# NBME project : Su Han Cho

In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchtext import datasets
from torchtext.legacy import data

In [2]:
LABEL = data.Field(sequential=False,batch_first=True)
ANNOTATION = data.Field(sequential=True,batch_first=True,lower=True)

In [3]:
field = [('a',ANNOTATION),('l',LABEL)]

In [4]:
train_data= data.TabularDataset.splits(path = '/Users/suhancho/data/kaggle/nbme/',
                                       train = 'train.tsv',
                                       format = 'tsv',
                                       fields = field,
                                       skip_header = True)[0]

In [5]:
ANNOTATION.build_vocab(train_data)
LABEL.build_vocab(train_data)

In [7]:
# 하이퍼파라미터
BATCH_SIZE = 64
lr = 0.005
EPOCHS = 15
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
print("다음 기기로 학습합니다:", DEVICE)


다음 기기로 학습합니다: cpu


In [8]:
train_data, val_data = train_data.split(split_ratio=0.8)
train_iter, val_iter= data.BucketIterator.splits(
        (train_data, val_data), batch_size=BATCH_SIZE,
        shuffle=True, repeat=False,sort=False)
vocab_size = len(ANNOTATION.vocab)
n_classes = 144

# Model building

In [10]:
class BasicGRU(nn.Module):
    def __init__(self, n_layers, hidden_dim, n_vocab, embed_dim, n_classes, dropout_p=0.2):
        super(BasicGRU, self).__init__()
        print("Building Basic GRU model...")
        self.n_layers = n_layers
        self.embed = nn.Embedding(n_vocab, embed_dim)
        self.hidden_dim = hidden_dim
        self.dropout = nn.Dropout(dropout_p)
        self.gru = nn.GRU(embed_dim, self.hidden_dim,
                          num_layers=self.n_layers,
                          batch_first=True)
        self.out = nn.Linear(self.hidden_dim, n_classes)
    
    def _init_state(self, batch_size=1):
        weight = next(self.parameters()).data
        return weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()
    
    def forward(self, x):
        x = self.embed(x)
        h_0 = self._init_state(batch_size=x.size(0))
        x, _ = self.gru(x, h_0)  # [i, b, h]
        h_t = x[:,-1,:]
        self.dropout(h_t)
        logit = self.out(h_t)  # [b, h] -> [b, o]
        return logit


In [11]:
def train(model, optimizer, train_iter):
    model.train()
    for b, batch in enumerate(train_iter):
        x, y = batch.a.to(DEVICE), batch.l.to(DEVICE)
        optimizer.zero_grad()
        logit = model(x)
        loss = F.cross_entropy(logit, y)
        loss.backward()
        optimizer.step()

In [12]:
def evaluate(model, val_iter):
    """evaluate model"""
    model.eval()
    corrects, total_loss = 0, 0
    for batch in val_iter:
        x, y = batch.a.to(DEVICE), batch.l.to(DEVICE)
        logit = model(x)
        loss = F.cross_entropy(logit, y, reduction='sum')
        total_loss += loss.item()
        corrects += (logit.max(1)[1].view(y.size()).data == y.data).sum()
    size = len(val_iter.dataset)
    avg_loss = total_loss / size
    avg_accuracy = 100.0 * corrects / size
    return avg_loss, avg_accuracy

In [13]:

# n_layers, hidden_dim, n_vocab, embed_dim, n_classes, dropout_p=0.2)
model = BasicGRU(1, 128, vocab_size, 256, n_classes, 0.2).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

Building Basic GRU model...


In [15]:
best_val_loss = None
for e in range(1, EPOCHS+1):
    train(model, optimizer, train_iter)
    val_loss, val_accuracy = evaluate(model, val_iter)

    print("[Epoch: %d] Validation Error : %5.2f | Validation Accuracy : %5.2f" % (e, val_loss, val_accuracy))
    
    # 검증 오차가 가장 적은 최적의 모델을 저장
    if not best_val_loss or val_loss < best_val_loss:
        if not os.path.isdir("snapshot"):
            os.makedirs("snapshot")
        torch.save(model.state_dict(), './snapshot/txtclassification.pt')
        best_val_loss = val_loss

[Epoch: 1] Validation Error :  0.75 | Validation Accuracy : 75.82
[Epoch: 2] Validation Error :  0.76 | Validation Accuracy : 75.00
[Epoch: 3] Validation Error :  0.67 | Validation Accuracy : 77.46
[Epoch: 4] Validation Error :  0.70 | Validation Accuracy : 76.45
[Epoch: 5] Validation Error :  0.69 | Validation Accuracy : 76.77
[Epoch: 6] Validation Error :  0.68 | Validation Accuracy : 77.84
[Epoch: 7] Validation Error :  0.70 | Validation Accuracy : 76.39
[Epoch: 8] Validation Error :  0.68 | Validation Accuracy : 77.59
[Epoch: 9] Validation Error :  0.70 | Validation Accuracy : 77.21
[Epoch: 10] Validation Error :  0.69 | Validation Accuracy : 78.09
[Epoch: 11] Validation Error :  0.67 | Validation Accuracy : 78.09
[Epoch: 12] Validation Error :  0.71 | Validation Accuracy : 76.26
[Epoch: 13] Validation Error :  0.71 | Validation Accuracy : 77.27
[Epoch: 14] Validation Error :  0.70 | Validation Accuracy : 77.34
[Epoch: 15] Validation Error :  0.71 | Validation Accuracy : 77.34


In [52]:
model(

BasicGRU(
  (embed): Embedding(2456, 256)
  (dropout): Dropout(p=0.2, inplace=False)
  (gru): GRU(256, 128, batch_first=True)
  (out): Linear(in_features=128, out_features=144, bias=True)
)

In [ ]:
model.load_state_dict(torch.load('./snapshot/txtclassification.pt'))
test_loss, test_acc = evaluate(model, test_iter)
print('테스트 오차: %5.2f | 테스트 정확도: %5.2f' % (test_loss, test_acc))